# Phase 5: LLM-Based Data Labeling
## Fill Missing Values Using Gemini 2.0 Flash API

This notebook uses Google's Gemini 2.0 Flash to fill missing values in the master training dataset (v5).

### Prerequisites:
- Run notebook 05 first to create `master_training_data_v5.csv` (60K stratified rows)

### What we're filling:

| Source | Missing Columns | Rows |
|--------|-----------------|------|
| Crisis tweets | emotion_label, emotion_name | ~25K |
| Non-crisis tweets | emotion_label, emotion_name | ~18K |
| GoEmotions | crisis_label, event_type, event_name, informativeness | ~17K |

### Strategy (per mentor's advice):
- Process **one row at a time** for better accuracy
- Each tweet gets its own API call
- Save progress periodically to handle interruptions



## 1. Setup and Imports

In [ ]:
# Install required packages if not already installed
# !pip install google-generativeai python-dotenv tqdm

import pandas as pd
import numpy as np
import google.generativeai as genai
import json
import time
import os
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✅ Libraries loaded successfully!")

## 2. Configure Gemini API

In [ ]:
# Load API key from .env file
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GEMINI_API_KEY or GEMINI_API_KEY == 'your_api_key_here':
    print("⚠️  Please set your Gemini API key in the .env file!")
    print("   1. Open .env file in your project root")
    print("   2. Replace 'your_api_key_here' with your actual API key")
    print("   3. Get a key at: https://makersuite.google.com/app/apikey")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Gemini API configured from .env file!")

In [ ]:
# Initialize the model
# Options: 
#   - "gemini-2.0-flash" (recommended - good balance of speed/quality)
#   - "gemini-2.0-flash-lite" (cheaper but lower accuracy)
#   - "gemini-1.5-flash" (previous gen, also good)
MODEL_NAME = "gemini-2.0-flash"

model = genai.GenerativeModel(
    model_name=MODEL_NAME,
    generation_config={
        "temperature": 0.1,  # Low temperature for consistent classification
        "top_p": 0.95,
        "max_output_tokens": 100,  # We only need short responses
    }
)

print(f"✅ Model initialized: {MODEL_NAME}")

## 3. Load Master Dataset (v5 - 60K stratified rows)

In [ ]:
# Load the stratified master training dataset from notebook 05
MASTER_PATH = 'master_training_data/master_training_data_v5.csv'

df = pd.read_csv(MASTER_PATH)
print(f"✅ Loaded {len(df):,} rows from {MASTER_PATH}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nNull counts:")
print(df.isnull().sum())
print(f"\nSource distribution:")
print(df['source_dataset'].value_counts())

In [ ]:
# Remove rows with null text (can't classify empty text)
null_text_count = df['text'].isna().sum()
if null_text_count > 0:
    print(f"Removing {null_text_count} rows with null text...")
    df = df[df['text'].notna()].reset_index(drop=True)
    print(f"Remaining rows: {len(df):,}")
else:
    print("✅ No null text rows found")

## 4. Define Classification Schema

In [ ]:
# Emotion mapping (label -> name)
EMOTION_MAP = {
    1: "admiration",
    2: "anger",
    3: "annoyance",
    4: "caring",
    5: "confusion",
    6: "curiosity",
    7: "desire",
    8: "disappointment",
    9: "fear",
    10: "gratitude",
    11: "joy",
    12: "amusement",
    13: "neutral"
}

# Reverse mapping (name -> label)
EMOTION_NAME_TO_LABEL = {v: k for k, v in EMOTION_MAP.items()}

# Informativeness labels (from CrisisLex)
INFORMATIVENESS_LABELS = [
    "related_informative",
    "related_not_informative",
    "not_related"
]

print("13 Emotion categories:")
for label, name in EMOTION_MAP.items():
    print(f"  {label}: {name}")

## 5. Prompt Templates

In [ ]:
# Prompt for classifying EMOTION (for Crisis/Non-crisis tweets)
EMOTION_PROMPT_TEMPLATE = """Classify the emotion in this tweet into exactly ONE of these 13 categories:
1. admiration - respect, approval for someone/something
2. anger - strong displeasure, hostility
3. annoyance - mild irritation, frustration
4. caring - concern, empathy for others
5. confusion - uncertainty, bewilderment
6. curiosity - desire to learn or know
7. desire - wanting something
8. disappointment - sadness from unmet expectations
9. fear - anxiety, worry, being scared
10. gratitude - thankfulness, appreciation
11. joy - happiness, excitement, celebration
12. amusement - finding something funny/entertaining
13. neutral - no strong emotion, factual statement

Tweet: "{text}"

Respond with ONLY the emotion name (one word, lowercase). Nothing else."""


# Prompt for classifying GoEmotions data (crisis_label, event_type, informativeness)
GOEMOTIONS_PROMPT_TEMPLATE = """Analyze this Reddit comment and determine:
1. Is it related to a crisis/disaster/emergency? (yes/no)
2. If crisis-related, what type? (hurricane, earthquake, flood, wildfire, terrorism, shooting, pandemic, other_crisis)
   If NOT crisis-related, use: general_discussion
3. Informativeness: 
   - "related_informative" = discusses crisis with useful info
   - "related_not_informative" = mentions crisis but no useful info
   - "not_related" = not about any crisis

Comment: "{text}"

Respond in this EXACT format (3 lines):
crisis: yes or no
event_type: <type>
informativeness: <label>"""

print("✅ Prompt templates defined")

## 6. Classification Functions

In [ ]:
def classify_emotion(text, max_retries=3):
    """Classify the emotion of a tweet using Gemini."""
    prompt = EMOTION_PROMPT_TEMPLATE.format(text=text[:500])
    
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            emotion_name = response.text.strip().lower()
            
            if emotion_name in EMOTION_NAME_TO_LABEL:
                return EMOTION_NAME_TO_LABEL[emotion_name], emotion_name
            else:
                for name in EMOTION_NAME_TO_LABEL:
                    if name in emotion_name:
                        return EMOTION_NAME_TO_LABEL[name], name
                return 13, "neutral"
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print(f"Error: {str(e)[:50]}")
                return None, None
    return None, None


def classify_goemotions_row(text, max_retries=3):
    """Classify a GoEmotions comment for crisis-related fields."""
    prompt = GOEMOTIONS_PROMPT_TEMPLATE.format(text=text[:500])
    
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            lines = response.text.strip().lower().split('\n')
            
            crisis_label = 0
            event_type = "general_discussion"
            informativeness = "not_related"
            
            for line in lines:
                if 'crisis:' in line:
                    crisis_label = 1 if 'yes' in line else 0
                elif 'event_type:' in line:
                    event_type = line.split(':')[1].strip()
                elif 'informativeness:' in line:
                    info = line.split(':')[1].strip()
                    if 'not_related' in info or 'not related' in info:
                        informativeness = 'not_related'
                    elif 'not_informative' in info:
                        informativeness = 'related_not_informative'
                    elif 'informative' in info:
                        informativeness = 'related_informative'
            
            event_name = event_type
            return crisis_label, event_type, event_name, informativeness
            
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print(f"Error: {str(e)[:50]}")
                return None, None, None, None
    return None, None, None, None

print("✅ Classification functions defined")

## 7. Test Classification (Verify API works)

In [ ]:
# Test emotion classification
print("Testing emotion classification...")
test1 = "Pray for the victims of the hurricane. So devastating."
result = classify_emotion(test1)
print(f"  '{test1[:50]}...'")
print(f"  → {result[1]} (label: {result[0]})")

test2 = "GOOOAAL! France wins the World Cup!"
result = classify_emotion(test2)
print(f"\n  '{test2}'")
print(f"  → {result[1]} (label: {result[0]})")

print("\nTesting GoEmotions classification...")
test3 = "This meme is hilarious, I can't stop laughing"
result = classify_goemotions_row(test3)
print(f"  '{test3}'")
print(f"  → crisis: {result[0]}, event_type: {result[1]}, informativeness: {result[3]}")

## 8. Identify Rows That Need Labeling

In [ ]:
# Identify rows needing emotion classification (Crisis + Non-crisis)
needs_emotion = df['emotion_label'].isna()
emotion_indices = df[needs_emotion].index.tolist()

# Identify GoEmotions rows needing crisis-related fields
is_goemotions = df['source_dataset'] == 'GoEmotions'
goemotions_indices = df[is_goemotions].index.tolist()

print(f"Rows needing emotion classification: {len(emotion_indices):,}")
print(f"GoEmotions rows needing crisis fields: {len(goemotions_indices):,}")
print(f"\nTotal API calls needed: {len(emotion_indices) + len(goemotions_indices):,}")

# Cost estimate
total_calls = len(emotion_indices) + len(goemotions_indices)
est_cost = (total_calls * 200 * 0.10 / 1_000_000) + (total_calls * 30 * 0.40 / 1_000_000)
print(f"Estimated cost: ${est_cost:.2f} USD")

## 9. Progress Tracking Setup

In [ ]:
# Create checkpoint directory
CHECKPOINT_DIR = Path('llm_labeling_checkpoints')
CHECKPOINT_DIR.mkdir(exist_ok=True)

CHECKPOINT_FILE = CHECKPOINT_DIR / 'labeled_progress.csv'
PROGRESS_FILE = CHECKPOINT_DIR / 'progress.json'

def load_progress():
    if PROGRESS_FILE.exists():
        with open(PROGRESS_FILE, 'r') as f:
            return json.load(f)
    return {'emotion_last_idx': -1, 'goemotions_last_idx': -1}

def save_progress(emotion_idx, goemotions_idx):
    with open(PROGRESS_FILE, 'w') as f:
        json.dump({
            'emotion_last_idx': emotion_idx,
            'goemotions_last_idx': goemotions_idx,
            'timestamp': datetime.now().isoformat()
        }, f)

progress = load_progress()
print(f"Loaded progress: {progress}")

## 10. Main Processing Loop - EMOTION CLASSIFICATION

Fill `emotion_label` and `emotion_name` for Crisis + Non-crisis tweets (~43K rows)

**Estimated time: 1-2 hours | Estimated cost: ~$1 USD**

In [ ]:
# Configuration
BATCH_SIZE = 100          # Save checkpoint every N rows
RATE_LIMIT_DELAY = 0.1    # Seconds between API calls
MAX_ROWS = None           # Set to a number for testing (e.g., 100), None for all

# Get starting point from checkpoint
start_idx = progress['emotion_last_idx'] + 1
indices_to_process = emotion_indices[start_idx:]

if MAX_ROWS:
    indices_to_process = indices_to_process[:MAX_ROWS]

print(f"Emotion classification config:")
print(f"  Start index: {start_idx}")
print(f"  Rows to process: {len(indices_to_process):,}")
print(f"  Batch size: {BATCH_SIZE}")

In [ ]:
# Process emotion classification
errors = []
processed = 0

for i, idx in enumerate(tqdm(indices_to_process, desc="Classifying emotions")):
    text = df.at[idx, 'text']
    
    if pd.isna(text) or str(text).strip() == '':
        continue
    
    emotion_label, emotion_name = classify_emotion(text)
    
    if emotion_label is not None:
        df.at[idx, 'emotion_label'] = emotion_label
        df.at[idx, 'emotion_name'] = emotion_name
        processed += 1
    else:
        errors.append(idx)
    
    time.sleep(RATE_LIMIT_DELAY)
    
    if (i + 1) % BATCH_SIZE == 0:
        save_progress(start_idx + i, progress['goemotions_last_idx'])
        df.to_csv(CHECKPOINT_FILE, index=False)

# Final save
save_progress(start_idx + len(indices_to_process) - 1, progress['goemotions_last_idx'])
df.to_csv(CHECKPOINT_FILE, index=False)

print(f"\n✅ Emotion classification complete!")
print(f"   Processed: {processed:,}")
print(f"   Errors: {len(errors)}")

## 11. Main Processing Loop - GOEMOTIONS CRISIS FIELDS

Fill `crisis_label`, `event_type`, `event_name`, `informativeness` for GoEmotions (~17K rows)

**Estimated time: 30-60 minutes | Estimated cost: ~$0.50 USD**

In [ ]:
# Reload progress
progress = load_progress()

start_idx = progress['goemotions_last_idx'] + 1
indices_to_process = goemotions_indices[start_idx:]

if MAX_ROWS:
    indices_to_process = indices_to_process[:MAX_ROWS]

print(f"GoEmotions classification config:")
print(f"  Start index: {start_idx}")
print(f"  Rows to process: {len(indices_to_process):,}")

In [ ]:
# Process GoEmotions rows
errors_goe = []
processed_goe = 0

for i, idx in enumerate(tqdm(indices_to_process, desc="Classifying GoEmotions")):
    text = df.at[idx, 'text']
    
    if pd.isna(text) or str(text).strip() == '':
        continue
    
    crisis_label, event_type, event_name, informativeness = classify_goemotions_row(text)
    
    if crisis_label is not None:
        df.at[idx, 'crisis_label'] = crisis_label
        df.at[idx, 'event_type'] = event_type
        df.at[idx, 'event_name'] = event_name
        df.at[idx, 'informativeness'] = informativeness
        processed_goe += 1
    else:
        errors_goe.append(idx)
    
    time.sleep(RATE_LIMIT_DELAY)
    
    if (i + 1) % BATCH_SIZE == 0:
        save_progress(progress['emotion_last_idx'], start_idx + i)
        df.to_csv(CHECKPOINT_FILE, index=False)

# Final save
save_progress(progress['emotion_last_idx'], start_idx + len(indices_to_process) - 1)
df.to_csv(CHECKPOINT_FILE, index=False)

print(f"\n✅ GoEmotions classification complete!")
print(f"   Processed: {processed_goe:,}")
print(f"   Errors: {len(errors_goe)}")

## 12. Validation - Check Results

In [ ]:
print("=" * 80)
print("VALIDATION - After LLM Labeling")
print("=" * 80)

print(f"\nNull counts:")
print(df.isnull().sum())

print(f"\nEmotion distribution:")
print(df['emotion_name'].value_counts())

print(f"\nCrisis label distribution:")
print(df['crisis_label'].value_counts())

print(f"\nEvent type distribution:")
print(df['event_type'].value_counts())

## 13. Save Final Labeled Dataset

In [ ]:
# Save final labeled dataset
OUTPUT_PATH = 'master_training_data/master_training_data_v5_labelled.csv'

df.to_csv(OUTPUT_PATH, index=False)

file_size = Path(OUTPUT_PATH).stat().st_size / (1024**2)

print("=" * 80)
print("FINAL LABELED DATASET SAVED")
print("=" * 80)
print(f"\n✅ Saved to: {OUTPUT_PATH}")
print(f"   File size: {file_size:.2f} MB")
print(f"   Total rows: {len(df):,}")
print(f"\nAll rows now have complete labels for BERT training!")

In [ ]:
# Create sample file
SAMPLE_PATH = 'master_training_data/master_training_sample_v5_labelled.csv'
df_sample = df.sample(n=min(10000, len(df)), random_state=42)
df_sample.to_csv(SAMPLE_PATH, index=False)

print(f"✅ Created sample file: {SAMPLE_PATH}")
print(f"   Rows: {len(df_sample):,}")

## 14. Final Summary

In [ ]:
print("=" * 80)
print("LLM LABELING COMPLETE - FINAL SUMMARY")
print("=" * 80)

print(f"\n📊 Dataset Statistics:")
print(f"   Total rows: {len(df):,}")
print(f"   Rows with emotion labels: {df['emotion_label'].notna().sum():,}")
print(f"   Rows with crisis labels: {df['crisis_label'].notna().sum():,}")

print(f"\n📁 Files Created:")
print(f"   Main: {OUTPUT_PATH}")
print(f"   Sample: {SAMPLE_PATH}")

print(f"\n✅ Dataset is now ready for multi-task BERT training!")
print(f"   - All rows have emotion_label and emotion_name")
print(f"   - All rows have crisis_label")
print(f"   - All rows have event_type and informativeness")
print("=" * 80)